In [86]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

X = pd.read_csv("../input/cimis-dataset-with-fire-target/all_conditions.csv")
X_test_full = pd.read_csv('../input/cimis-dataset-with-fire-target/all_conditions.csv', index_col="Stn Id")
X.dropna(axis=0, subset=['Target'], inplace=True)
X = X.dropna().reset_index()
X = X.drop(["Stn Name", "CIMIS Region", "Date", "index", "Stn Id", "ETo (in)", "Max Air Temp (F)", 'Min Air Temp (F)', 'Max Rel Hum (%)', 'Wind Run (miles)'], axis=1)
y = X.pop("Target")

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)
# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numeric columns
numeric_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

X.columns


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [87]:
from xgboost import XGBClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import cross_val_score

model = XGBClassifier(learning_rate=0.2, max_depth=6, n_estimators=500, random_state=1)



model.fit(X_train_full, y_train)
predictions = model.predict(X_valid_full)



scores = cross_val_score(model, X, y, cv=5, scoring="neg_mean_absolute_error")
print("MAE scores:\n", scores)

#print(mean_absolute_error(predictions, y_valid))

In [76]:
from sklearn.model_selection import GridSearchCV

params = { 'max_depth': [6, 8, 10],
           'learning_rate': [0.1],
           'n_estimators': [500, 600, 700]}
grid_search = GridSearchCV(estimator=model, 
                   param_grid=params,
                   scoring='neg_root_mean_squared_error', 
                   verbose=1)
grid_search.fit(X_train_full, y_train)
print("Best parameters:", grid_search.best_params_)
print("Lowest RMSE: ", (grid_search.best_score_))


In [ ]:
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (10,6)
plt.scatter(X["Stn Id"], X["Avg Wind Speed (mph)"])
plt.show()